In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("../data_given/winequality.csv")

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,TARGET
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'TARGET'],
      dtype='object')

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [7]:
overview = df.describe()

In [9]:
overview.loc[["min", "max"]].to_dict()

{'fixed acidity': {'min': 4.6, 'max': 15.9},
 'volatile acidity': {'min': 0.12, 'max': 1.58},
 'citric acid': {'min': 0.0, 'max': 1.0},
 'residual sugar': {'min': 0.9, 'max': 15.5},
 'chlorides': {'min': 0.012, 'max': 0.611},
 'free sulfur dioxide': {'min': 1.0, 'max': 72.0},
 'total sulfur dioxide': {'min': 6.0, 'max': 289.0},
 'density': {'min': 0.99007, 'max': 1.00369},
 'pH': {'min': 2.74, 'max': 4.01},
 'sulphates': {'min': 0.33, 'max': 2.0},
 'alcohol': {'min': 8.4, 'max': 14.9},
 'TARGET': {'min': 3.0, 'max': 8.0}}

In [10]:
overview.loc[["min", "max"]].to_json("schema_in.json")

In [13]:
#Custom error 

class NotInRangeError(Exception):
    def __init__(self, message="column value not in acceptable range"):
        
        self.message = message
        super().__init__(self.message)

In [14]:
raise NotInRangeError

NotInRangeError: column value not in acceptable range

In [6]:
pwd

'/home/abhishek/local_disk/programming/mlops/Ml_ops_project/project1/notebooks'

In [7]:
os.chdir('/home/abhishek/local_disk/programming/mlops/Ml_ops_project/project1/notebooks')

In [11]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]

    mlflow.set_registry_uri(remote_server_uri)
    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.trasition_model_version_stage(
                name = model_name,
                version = current_version,
                stage="Production"
            )
        else:
            current_version = mv["version"]
            client.trasition_model_version_stage(
                name = model_name,
                version = current_version,
                stage="Staging"
            )

log_production_model("../params.yaml")


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'int'